# 第10课：Actor-Critic 方法（Actor-Critic Methods）

> policy-based plus value-based

one view —— incorporates both policy-based and value-based methods
- “actor” refers to a policy update step
- “critic” refers to a value update step

another view —— actor-critic methods are still policy gradient algorithms

## Part 1: 最简单的 Actor-Critic（QAC）

- The critic corresponds to the value update step via the Sarsa algorithm.
- The actor corresponds to the policy update step.
- This actor-citric algorithm is sometimes called Q actor-critic (QAC).

---

### 核心思想：策略 + 值函数的协同学习

Actor-Critic 方法融合了两大类强化学习方法：

| 类型         | 作用                               |             |
| ---------- |----------------------------------| ----------- |
| **Actor**  | 表示策略 $\pi\_\theta(a\| s)$，用于选择动作 |
| **Critic** | 表示值函数 $V\_w(s)$ 或 $Q\_w(s,a)$，用于评估 Actor 的行为 |             |

---

### 最简单的形式：QAC（Q-based Actor-Critic）

QAC 使用 **动作值函数** $Q_w(s,a)$ 作为 Critic，来对策略 $\pi_\theta(a|s)$ 进行评估与更新。

---

### 结构描述

* **Actor（策略）**：

  * 输入状态 $s$，输出每个动作的概率 $\pi_\theta(a|s)$
* **Critic（动作值函数）**：

  * 输入 $(s,a)$，输出动作值 $Q_w(s,a)$

---

###  策略梯度更新（使用 Critic 提供的 \$Q\$ 值）

策略梯度估计公式：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{s \sim d^\pi,\, a \sim \pi_\theta} \left[ \nabla_\theta \log \pi_\theta(a|s) \cdot Q_w(s, a) \right]
$$

---

###  Critic 的 TD 更新方式

计算 TD 误差：

$$
\delta_t = r_{t+1} + \gamma Q_w(s_{t+1}, a_{t+1}) - Q_w(s_t, a_t)
$$

更新 Critic 参数 $w$（最小化 TD 误差的平方）：

$$
w \leftarrow w - \beta \cdot \nabla_w \left( \delta_t^2 \right)
$$

或使用梯度更新形式：

$$
w \leftarrow w + \beta \cdot \delta_t \cdot \nabla_w Q_w(s_t, a_t)
$$

---

![](./img/10_1.png)

###  算法流程（简化）

1. 初始化策略参数 $\theta$ 和值函数参数 $w$
2. 重复直到收敛：

   对每一步 $(s\_t, a\_t, r\_{t+1}, s\_{t+1})$：

   * 使用 $\pi_\theta(a_t|s_t)$ 选择动作
   * 与环境交互，获得奖励 $r_{t+1}$ 和下一状态 $s_{t+1}$
   * Critic 使用 TD 方法更新 $Q_w$
   * Actor 使用策略梯度更新 $\theta$

---

###  Actor 与 Critic 的协同机制

| 模块     | 输入       | 输出        | 作用               |                     |
| ------ | -------- |-----------|------------------| ------------------- |
| Actor  | 状态 $s$   | 策略 $\pi(a\|s)$             | 采样动作，使用 $Q$ 值优化策略 |
| Critic | $(s, a)$ | $Q(s, a)$ | TD 学习，提供 Actor 更新目标 |                     |

---


###  特点总结

| 优点                   | 缺点                    |
| -------------------- | --------------------- |
| 同时学习策略和值函数，融合策略法与价值法 | 同时优化两个网络，超参数敏感        |
| TD 更新高效，适用于持续任务      | 值函数估计不准时，会导致策略更新不稳定   |
| 相比 REINFORCE 方差更低    | 策略网络与值网络需要尺度匹配，结构设计复杂 |






## Part 2: Advantage Actor-Critic (A2C)

A2C 是将 Actor-Critic 和 advantage function 结合起来，接入一个更 stable 、无偏的目标。

### 一. 加入 Advantage 的助益

在前置的 QAC 中：

$$
\nabla_\theta J(\theta) \propto \nabla_\theta \log \pi_\theta(a|s) Q_w(s, a)
$$

但 Q-value 常带有大的偏差和无关情报，影响改进方向。所以我们展开出 Advantage function：

$$
A(s,a) = Q(s,a) - V(s)
$$

实际上代替 Q(s,a) 的是下面这个漏动项：

$$
\nabla_\theta J(\theta) \propto \nabla_\theta \log \pi_\theta(a|s) (Q_w(s,a) - V_v(s))
$$

---

### 二. A2C 算法流程

![](./img/10_2.png)

1. 初始化策略参数 \$\theta\$，Critic 的状态值网络参数 \$v\$，动作值网参数 \$w\$
2. 对于每一步 \$(s\_t,a\_t,r\_{t+1},s\_{t+1})\$：

   * 根据 \$\pi\_\theta(a|s\_t)\$ 采样动作 \$a\_t\$
   * Critic 估计出 \$V\_v(s\_t), V\_v(s\_{t+1})\$
   * 计算 TD target: \$r\_{t+1} + \gamma V\_v(s\_{t+1})\$
   * Advantage 预估：\$A\_t = r\_{t+1} + \gamma V\_v(s\_{t+1}) - V\_v(s\_t)\$
   * 策略更新：

     $$
     \theta \leftarrow \theta + \alpha \cdot \nabla_\theta \log \pi_\theta(a_t|s_t) \cdot A_t
     $$
   * 值网更新：

     $$
     v \leftarrow v - \beta \cdot \nabla_v \left( A_t^2 \right)
     $$

---

### 三. A2C 的特点

| 优点                    | 解释                              |
| --------------------- | ------------------------------- |
| Advantage 降低了偏差       | Q 值带有偏差，A 用 V 帮助筛除地方或无关事项       |
| TD + Advantage 带来快速收敛 | 用尽可能小的 bootstrap step 更新 Critic |
| 更 stable 的策略更新        | 目标函数被 baseline 系统地控制，削减无关方向     |

---

A2C 常被用作新型策略梯升策略学习算法的基础，是实际 Actor-Critic 算法中的重要基石。




## Part 3：重要性采样和 Off-policy Actor-Critic

---

### 1. 背景介绍



在强化学习中，**On-policy**算法(REINFORCE, QAC, and A2C)要求策略更新时必须使用当前策略生成的数据，这限制了数据的重复利用效率。相比之下，**Off-policy**算法允许使用与当前策略不同的行为策略（Behavior Policy）生成的数据进行学习，极大提升了样本效率。

然而，Off-policy学习带来了分布偏差（distribution mismatch）的问题，即数据来自的行为策略与目标策略不同，直接使用这些数据会导致估计偏差和不稳定。

---

### 2. Off-policy Actor-Critic 面临的挑战

* **行为策略与目标策略不同**：数据采样分布是行为策略 $\mu(a|s)$，而目标策略是 $\pi_\theta(a|s)$。
* **策略梯度估计的偏差**：若直接用行为策略数据计算目标策略的梯度，会引入偏差。
* **价值函数估计的偏差和方差**：因数据分布不同，估计值函数的收敛和准确性变差。

---

### 3. 重要性采样（Importance Sampling）

重要性采样是一种统计技术，用来纠正样本分布不一致导致的偏差。

#### 3.1 基本原理

对于期望 $\mathbb{E}_{a \sim \pi} [f(a)]$，若数据采样自分布 $\mu$，则：

$$
\mathbb{E}_{a \sim \pi}[f(a)] = \mathbb{E}_{a \sim \mu}\left[ \frac{\pi(a)}{\mu(a)} f(a) \right]
$$

其中，权重比

$$
w(a) = \frac{\pi(a)}{\mu(a)}
$$

称为**重要性权重**，用来校正分布差异。

---

#### 3.2 在 Actor-Critic 策略梯度中的应用

策略梯度期望原本是

$$
\nabla_\theta J(\theta) = \mathbb{E}_{s \sim d^\pi, a \sim \pi_\theta} \left[ \nabla_\theta \log \pi_\theta(a|s) Q^\pi(s,a) \right]
$$

如果采样数据来自行为策略 $\mu(a|s)$，则改写为：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{s \sim d^\mu, a \sim \mu} \left[ w(s,a) \cdot \nabla_\theta \log \pi_\theta(a|s) Q^\pi(s,a) \right]
$$

其中，

$$
w(s,a) = \frac{\pi_\theta(a|s)}{\mu(a|s)}
$$

---

#### 3.3 权重截断和方差控制

* 直接使用重要性权重会导致**高方差**，因为权重可能极大或极小。

* 常用的做法是**截断（clipping）权重**：

  $$
  \tilde{w}(s,a) = \min\left(c, \frac{\pi_\theta(a|s)}{\mu(a|s)}\right)
  $$

  其中，$c$ 是超参数，防止权重过大。

* 还可结合 **基于方差的归一化** 或 **去偏估计** 等技术进一步稳定训练。

---

### 4. Off-policy Actor-Critic 的算法设计

---

#### 4.1 基本框架

* **行为策略** $\mu(a|s)$：用于采样生成数据的策略，可以是旧策略或者专门的探索策略。
* **目标策略** $\pi_\theta(a|s)$：需要优化的策略。
* **Critic**：估计目标策略的价值函数，如动作值函数 $Q_w(s,a)$。

---

#### 4.2 训练流程

![](./img/10_3.png)


1. **采样阶段**
   由行为策略 $\mu(a|s)$ 与环境交互，收集数据 ${(s_t, a_t, r_{t+1}, s_{t+1})}$。

2. **Critic 更新**
   通过目标策略估计的 TD 误差进行更新（通常使用目标策略进行价值评估）：

   $$
   \delta_t = r_{t+1} + \gamma \mathbb{E}_{a' \sim \pi_\theta(\cdot|s_{t+1})} [Q_w(s_{t+1}, a')] - Q_w(s_t, a_t)
   $$

3. **Actor 更新**
   利用重要性采样权重对采样数据加权，估计策略梯度：

   $$
   \nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{t=1}^N w_t \cdot \nabla_\theta \log \pi_\theta(a_t|s_t) Q_w(s_t, a_t)
   $$

   其中，

   $$
   w_t = \frac{\pi_\theta(a_t|s_t)}{\mu(a_t|s_t)}
   $$

---

#### 4.3 Off-policy Actor-Critic 代表算法

* **Deterministic Policy Gradient (DPG)**
  目标策略为确定性策略，更新时利用行为策略数据和重要性采样。

* **Deep Deterministic Policy Gradient (DDPG)**
  结合深度网络和经验回放，使用行为策略采样数据和目标网络稳定训练。

* **Twin Delayed DDPG (TD3)**
  通过双重Critic和延迟更新等技巧提升稳定性。

* **Soft Actor-Critic (SAC)**
  结合最大熵策略目标，使用重要性采样思想，支持 Off-policy 学习。

---

### 5. 重要性采样的局限与改进

* **方差大**：高权重样本导致梯度估计方差增大，影响训练稳定。
* **权重估计依赖策略估计准确性**：策略参数更新后，权重也需及时更新，否则估计不准。
* **截断或归一化方法**：有效控制权重范围，降低方差。
* **基于重参数化的 Off-policy 方法**：如 DDPG、SAC，避免显式计算权重，提升稳定性。

---

### 6. 总结

| 方面       | 内容                                     |
| -------- | -------------------------------------- |
| **目的**   | 解决 Off-policy 强化学习中策略与行为策略分布不匹配的问题     |
| **关键技术** | 重要性采样权重，权重截断，策略梯度加权，稳定更新               |
| **优点**   | 允许利用旧数据或其他策略采样数据，提高样本利用率               |
| **挑战**   | 权重方差大，估计偏差，训练不稳定，需要设计稳定的权重校正方法         |
| **代表算法** | DDPG、TD3、SAC 等，结合深度网络和 Off-policy 学习框架 |

---

### 7. 进一步阅读与实践建议

* 理解**重要性采样**在强化学习中的理论基础。
* 深入学习**DDPG、TD3、SAC**等 Off-policy Actor-Critic 算法的源码实现。
* 探索**权重截断**和**方差归一化**方法对训练稳定性的影响。
* 结合**经验回放**机制，提升样本的重用能力。
* 关注新兴 Off-policy 策略优化方法，尝试将重要性采样与其他方法结合。





## Part 4：Deterministic Actor-Critic (DPG)

---

### 1. 背景介绍

传统的 Actor-Critic 方法通常使用**随机策略（Stochastic Policy）**，即策略输出的是动作的概率分布 $\pi_\theta(a|s)$，动作是随机采样的。这种方法虽然理论完善，但在**连续动作空间**中，采样动作的方差较大，导致策略梯度估计不稳定，收敛速度慢。

为此，**Deterministic Policy Gradient (DPG)** 方法提出使用**确定性策略（Deterministic Policy）**：

$$
\mu_\theta: \mathcal{S} to \mathcal{A}
$$

即策略直接输出动作值，而不是动作概率分布。这极大减少了策略梯度的方差，提升了效率。

---

### 2. 确定性策略定义与目标

* **确定性策略** $\mu_\theta(s)\$ 直接给出状态 $s$ 下的确定动作 $a$。
* 优化目标是最大化确定性策略下的期望累积奖励：

$$
J(\theta) = \mathbb{E}_{s \sim \rho^{\mu}} \left[ Q^{\mu}(s, \mu_\theta(s)) \right]
$$

其中，$\rho^{\mu}$ 是状态分布（由策略 $\mu_\theta$ 诱导的状态访问频率），$Q^\mu(s,a)$ 是在策略 $\mu$ 下的动作值函数。

---

### 3. DPG 策略梯度定理

与随机策略梯度不同，DPG 有专门的梯度表达式：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{s \sim \rho^\mu} \left[ \nabla_\theta \mu_\theta(s) \nabla_a Q^\mu(s, a) \big|_{a=\mu_\theta(s)} \right]
$$

**解释**：

* 策略梯度不再涉及 \$\log \pi\_\theta\$，而是通过链式法则对动作值函数对动作的梯度乘以策略对参数的梯度进行优化。
* 减少了采样动作的随机性导致的梯度估计方差。

---

### 4. DPG 算法结构

---

#### 4.1 Actor 网络

* 输入状态 \$s\$
* 输出确定动作 \$a = \mu\_\theta(s)\$

---

#### 4.2 Critic 网络

* 输入 \$(s,a)\$
* 输出动作值函数估计 \$Q\_w(s,a)\$

---

#### 4.3 更新步骤

1. **Critic 更新**

利用贝尔曼方程的 TD 误差训练 \$Q\_w\$：

$$
\delta_t = r_{t+1} + \gamma Q_w(s_{t+1}, \mu_\theta(s_{t+1})) - Q_w(s_t, a_t)
$$

更新参数 \$w\$：

$$
w \leftarrow w + \beta \delta_t \nabla_w Q_w(s_t, a_t)
$$

2. **Actor 更新**

利用 DPG 策略梯度公式：

$$
\nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{t=1}^N \nabla_\theta \mu_\theta(s_t) \nabla_a Q_w(s_t, a) \big|_{a=\mu_\theta(s_t)}
$$

更新 \$\theta\$：

$$
\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)
$$

---

### 5. DPG 算法伪代码

```pseudo
Initialize actor network μ_θ and critic network Q_w with random weights θ, w
Initialize replay buffer R

for each episode do
    Initialize state s
    for each step do
        Select action a = μ_θ(s) + noise  # 添加探索噪声
        Execute action a, observe reward r and next state s'
        Store (s, a, r, s') in replay buffer R

        Sample mini-batch from R:
            Compute target: y = r + γ Q_w(s', μ_θ(s'))
            Update critic by minimizing (Q_w(s,a) - y)^2
            Update actor using policy gradient:
                ∇_θ J ≈ ∇_θ μ_θ(s) ∇_a Q_w(s,a) |_{a=μ_θ(s)}

        s ← s'
```

---

### 6. DPG 的特点和优势

| 优点                | 说明                            |
| ----------------- | ----------------------------- |
| 减少策略梯度估计的方差       | 直接输出确定动作，避免随机策略中采样带来的方差       |
| 适合高维连续动作空间        | 避免了对整个动作空间求和/积分的问题            |
| 样本效率高             | 可以结合经验回放和离线数据进行 Off-policy 学习 |
| 可结合深度网络（DDPG）进行扩展 | 实现深度确定性策略梯度，能解决复杂控制任务         |

---

### 7. DPG 存在的挑战

* **局部极值风险**：确定性策略缺乏探索能力，易陷入局部最优。
* **探索策略设计困难**：需要额外设计噪声（如 Ornstein-Uhlenbeck 噪声）来探索环境。
* **依赖准确的 Critic 估计**：Critic 误差会严重影响 Actor 更新质量。
* **训练不稳定**：需要合理设计网络结构、超参数和目标网络。

---

### 8. 深度确定性策略梯度（DDPG）

DPG 是 DDPG 的理论基础，DDPG 结合了 DPG 的确定性策略和深度神经网络，并引入经验回放和目标网络，极大提高训练稳定性。

**DDPG 主要改进点**：

* **经验回放**：打破数据相关性，提高样本效率。
* **目标网络**：通过缓慢更新目标网络减少训练震荡。
* **探索噪声**：通过 Ornstein-Uhlenbeck 过程实现时间相关的动作噪声。

---

### 9. 总结

| 内容       | 说明                                                                                                     |
| -------- | ------------------------------------------------------------------------------------------------------ |
| **策略类型** | 确定性策略 \$\mu\_\theta(s)\$，直接输出动作                                                                        |
| **策略梯度** | \$\nabla\_\theta J = \mathbb{E}*{s} \left\[ \nabla*\theta \mu\_\theta(s) \nabla\_a Q\_w(s,a) \right]\$ |
| **适用场景** | 连续动作空间，尤其是高维连续动作的控制任务                                                                                  |
| **核心优势** | 减少策略梯度方差，提高训练效率                                                                                        |
| **关键难点** | 探索机制设计、稳定训练、Critic 准确估计                                                                                |

---

如果你需要，我还可以帮你继续写 DDPG 的代码示例、探索噪声的数学模型，或者结合 SAC、TD3 的对比讲解。需要吗？



# Q1 当前主流的强化学习方法是哪一类？value-based or policy-based？

这是一个非常好的问题，值得从多个维度来回答。目前（截至2025年），**强化学习领域的主流方法已经超越了传统的“value-based vs policy-based”二分法**，进入了融合与集成的时代，但我们仍可以根据你的问题来系统地梳理主流趋势：

---

##  一、从“学术研究热点”看主流方法

###  主流趋势是 **policy-based 方法为主，融合 value-based 的 Actor-Critic 架构**

现代强化学习研究中，尤其是在**连续控制、复杂策略优化**等任务上，**基于策略的方法（policy-based）已经成为主流**，尤其是以下几类：

| 方法类别                 | 主流算法/框架                  | 特点                          |
| -------------------- | ------------------------ | --------------------------- |
| **Policy-based**     | PPO, TRPO, REINFORCE     | 直接优化策略，收敛速度快，适合高维动作空间       |
| **Actor-Critic（融合）** | A2C, A3C, DDPG, TD3, SAC | 同时学习策略和值函数，兼顾收敛速度和样本效率      |
| **Value-based**      | DQN, Double DQN, Rainbow | 离散动作空间效果好，简单、稳定，但在复杂任务上表现受限 |

---

##  二、从“工业应用”看主流方法

在 **机器人控制、游戏 AI、自动驾驶、推荐系统** 等工业领域：

* **连续控制（机器人、自动驾驶）**：主流方法是 **policy-based + actor-critic**，如 DDPG、TD3、SAC。
* **大规模离散决策（推荐系统、广告）**：部分仍使用 DQN，但也在向 policy-based 方法过渡，配合 actor-critic。
* **大模型调度（如 AutoGPT、世界模型）**：更倾向使用模型预测 + policy gradient 优化。

---

##  三、从“强化学习顶会论文”趋势看

根据 NeurIPS / ICLR / ICML 等近几年论文统计：

* **SAC（Soft Actor-Critic）** 和 **PPO（Proximal Policy Optimization）** 是最常作为 baseline 的主流算法。
* 越来越多方法是基于 **actor-critic 架构**，将 policy-based 和 value-based 优点结合起来。
* Model-based RL、offline RL、RLHF 等领域方法大多以 **policy-based 框架为核心**。

---

## 总结结论：

| 问题维度 | 主流方法趋势                                        |
| ---- | --------------------------------------------- |
| 学术研究 | **policy-based + actor-critic** 为主流           |
| 工业落地 | 连续控制任务使用 policy-based，离散任务仍用 value-based，但在融合 |
| 算法代表 | PPO, SAC, DDPG, TD3 > DQN                     |
| 总体趋势 | **融合性方法 dominate：Actor-Critic 成为主干架构**        |

---

###  小结一句话：

> 当前主流强化学习方法是以 **policy-based 方法为核心、融合 value-based 的 actor-critic 架构**，如 PPO、SAC、TD3 等，具备样本效率、收敛性与可扩展性兼顾的优点。

